# Coindexed Profuctor Optics
###### (https://oleg.fi/gists/posts/2021-01-04-coindexed-optics.html)

This lays out the construction of basic profunctor optics, indexed optics, and then co-indexed optics. We'll deal with indices by using n-ary sums and products ala generic-sop.

In [81]:
:ext GADTs
:ext PolyKinds
:ext DataKinds
:ext TypeOperators
:ext DeriveFunctor

import Data.Kind

newtype I a = I { unI :: a } deriving (Show, Functor)
instance Applicative I where
  pure a = I a
  I f <*> I a = I $ f a

data NP :: (k -> Type) -> [k] -> Type where
  Nil :: NP f '[]
  (:*) :: f x -> NP f xs -> NP f (x ': xs)
infixr 4 :*
  
data NS :: (k -> Type) -> [k] -> Type where
  Z :: f x -> NS f (x ': xs)
  S :: NS f xs -> NS f (x ': xs)

Here we have the type of our optic which keeps indices in a type level lists.

In [73]:
type Optic p (is :: [Type]) (js :: [Type]) s t a b = p is a b -> p js s t

class Profunctor (p :: [Type] -> Type -> Type -> Type) where
  dimap :: (x -> a) -> (b -> y) -> p is a b -> p is x y
  
class Profunctor p => Mapping p where
  roam :: ((a -> b) -> s -> t) -> p is a b -> p is s t
  
mapped :: (Mapping p, Functor f)
       => Optic p is is (f a) (f b) a b
mapped = roam fmap

newtype FunArrow (is :: [Type]) a b =
  FunArrow { runFunArrow :: a -> b }
  
instance Profunctor FunArrow where
  dimap f g (FunArrow k) = FunArrow (g . k . f)
  
instance Mapping FunArrow where
  roam f (FunArrow k) = FunArrow $ f k
  
over :: Optic FunArrow is js s t a b
     -> (a -> b)
     -> s -> t
over o f = runFunArrow . o $ FunArrow f

over mapped succ [1,2,3]

[2,3,4]

## Indexed optics

In [74]:
:ext FunctionalDependencies

class Functor f => FunctorWithIndex i f | f -> i where
  imap :: (i -> a -> b) -> f a -> f b
  
-- lists are indexed by Nats
instance FunctorWithIndex Word [] where
  imap f = zipWith f [0..]
  
class Mapping p => IMapping p where
  iroam :: ((i -> a -> b) -> s -> t) -> p (i ': is) a b -> p is s t
  
imapped :: (FunctorWithIndex i f, IMapping p)
        => p (i ': is) a b -> p is (f a) (f b)
imapped = iroam imap

-- ignore the instance argument for FunArrows
instance IMapping FunArrow where
  iroam f (FunArrow k) = FunArrow . f $ const k
  
-- this is an arrow that takes a heterogenous list of indices in addition to it's argument
newtype IxFunArrow is a b =
  IxFunArrow { runIxFunArrow :: (NP I is, a) -> b }
  
instance Profunctor IxFunArrow where
  dimap f g (IxFunArrow k) = IxFunArrow $ g . k . fmap f
  
instance Mapping IxFunArrow where
  roam f (IxFunArrow k) = IxFunArrow $ \(i, s) -> f (curry k i) s
  
instance IMapping IxFunArrow where
  iroam f (IxFunArrow k) = IxFunArrow $ \(is, s) -> f (\i a -> k (I i :* is, a)) s
  
-- The indexed over. The general variant takes an optic with any indices list
gen_iover :: Optic IxFunArrow is '[] s t a b
          -> ((NP I is, a) -> b)
          -> s -> t
gen_iover o f s = runIxFunArrow (o $ IxFunArrow f) (Nil, s)

-- the single index variant
iover :: Optic IxFunArrow '[i] '[] s t a b
      -> (i -> a -> b)
      -> s -> t
iover o f s = runIxFunArrow (o $ IxFunArrow (\(I i :* Nil, a) -> f i a)) (Nil, s)

-- double index variant
iover2 :: Optic IxFunArrow '[i, j] '[] s t a b
       -> (i -> j -> a -> b)
       -> s -> t
iover2 o f = gen_iover o (\(I i :* I j :* Nil, a) -> f i j a)

iover imapped (\i -> if odd i then succ else id) [1,2,3,4,5]

[1,3,3,5,5]

## Coindexed optics

In [93]:
:ext RankNTypes
:ext EmptyCase
import Data.Maybe

class Functor f => FunctorWithCoindex j f | f -> j where
  jmap :: (a -> Either j b) -> f a -> f b
  
class (Traversable f, FunctorWithCoindex j f) => TraversableWithCoindex j f | f -> j where
  jtraverse :: Applicative m => (a -> m (Either j b)) -> f a -> m (f b)
  
-- the coindex of a list is ()
instance FunctorWithCoindex () [] where
  jmap f = mapMaybe (either (\() -> Nothing) Just . f)
  
instance TraversableWithCoindex () [] where
  jtraverse _ [] = pure []
  jtraverse f xs = fmap (jmap id) $ traverse f xs
  
  
class Profunctor p => Traversing p where
  wander :: (forall f. Applicative f => (a -> f b) -> s -> f t)
         -> p js a b -> p js s t
         
class Traversing p => JTraversing p where
  jwander :: (forall f. Applicative f => (a -> f (Either j b)) -> s -> f t)
          -> p (j ': js) a b -> p js s t
          
traversed :: (Traversable f, Traversing p) => Optic p js js (f a) (f b) a b
traversed = wander traverse

jtraversed :: (TraversableWithCoindex j f, JTraversing p)
           => Optic p (j ': js) js (f a) (f b) a b
jtraversed = jwander jtraverse

-- define a concrete profunctor to make use of it
newtype CoixFunArrow js a b =
  CoixFunArrow { runCoixFunArrow :: a -> Either (NS I js) b}
  
instance Profunctor CoixFunArrow where
  dimap f g (CoixFunArrow p) = CoixFunArrow (fmap g . p . f)
  
instance Traversing CoixFunArrow where
  wander f (CoixFunArrow p) = CoixFunArrow $ f p
  
instance JTraversing CoixFunArrow where
  jwander f (CoixFunArrow p) = CoixFunArrow $ f (plumb . p) where
    plumb :: Either (NS I (j ': js)) b -> Either (NS I js) (Either j b)
    plumb (Right x) = Right (Right x)
    plumb (Left (Z (I y))) = Right (Left y)
    plumb (Left (S z)) = Left z
      
nsAbsurd :: NS I '[] -> a
nsAbsurd x = case x of {}

gen_jover :: Optic CoixFunArrow js '[] s t a b
          -> (a -> Either (NS I js) b) -> s -> t
gen_jover o f = fmap (either nsAbsurd id) . runCoixFunArrow . o $ CoixFunArrow f

jover :: Optic CoixFunArrow '[i] '[] s t a b
      -> (a -> Either i b) -> s -> t
jover o f = gen_jover o (either (Left . Z . I) Right . f)

jover2 :: Optic CoixFunArrow '[i, j] '[] s t a b
       -> (a -> Either (Either i j) b)
       -> s -> t
jover2 o f = gen_jover o (go . f) where
  go (Left (Left i)) = Left . Z $ I i
  go (Left (Right j)) = Left . S . Z $ I j
  go (Right b) = Right b
  
jover jtraversed (\s -> if length s > 5 then Right s else Left ()) ["foobar", "xyzzy"]

["foobar"]